<a href="https://colab.research.google.com/github/jini1651/jongsul20/blob/main/Colaboratory%EC%97%90_%EC%98%A4%EC%8B%A0_%EA%B2%83%EC%9D%84_%ED%99%98%EC%98%81%ED%95%A9%EB%8B%88%EB%8B%A4%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
!pip install pefile

In [49]:
import os
from PIL import Image
import pefile
import csv

In [50]:
def getAddressfromPE(filename):
    startAddress_t, endAddress_t = 0, 0;
    startAddress_d, endAddress_d = 0, 0;
    startAddress_i, endAddress_i = 0, 0;
    startAddress_e, endAddress_e = 0, 0;

    try:
        pe = pefile.PE(filename)

        for section in pe.sections:
            if ".text" in str(section.Name):
                startAddress_t = section.VirtualAddress
                endAddress_t = section.next_section_virtual_address
            if ".data" in str(section.Name):
                startAddress_d = section.VirtualAddress
                endAddress_d = section.next_section_virtual_address
            if ".idata" in str(section.Name):
                startAddress_i = section.VirtualAddress
                endAddress_i = section.next_section_virtual_address
            if ".edata" in str(section.Name):
                startAddress_e = section.VirtualAddress
                endAddress_e = section.next_section_virtual_address

            break
    except pefile.PEFormatError:
        pass
    return startAddress_t, endAddress_t, startAddress_d, endAddress_d, startAddress_i, endAddress_i, startAddress_e, endAddress_e

In [51]:
def getBinaryData_t(filename, start, end):
    binaryValues = []
    file = open(filename, "rb")
    file.seek(start)
    data = file.read(1)
    i=0
    while i<65536: #256*256
        try:
          if data != b"" and file.tell() != end : 
            binaryValues.append(ord(data))
          else : 
            binaryValues.append(0)
        except TypeError:
            pass
        data = file.read(1)
        i=i+1
        

    print("'.text' data is produced.")

    return binaryValues

In [52]:
def getBinaryData_d(filename, start, end):
    binaryValues = []
    file = open(filename, "rb")
    file.seek(start)
    data = file.read(1)
    i=0
    while i<65536: #256*256
        try:
          if data != b"" and file.tell() != end :
            binaryValues.append(ord(data))
          else : 
            binaryValues.append(0)
        except TypeError:
            pass
        data = file.read(1)
        i=i+1

    print("'.data' data is produced.")
    
    return binaryValues

In [59]:
def getBinaryData_ie(filename, start1, end1, start2, end2):
  binaryValues = []
  file = open(filename, "rb")
  file.seek(start1)
  data = file.read(1)

  i=0
  while  i<32768: #128*256
    try:
      if data != b"" and file.tell() != end1 :
        binaryValues.append(ord(data))
      else :
         binaryValues.append(0)
    except TypeError:
      pass
    data = file.read(1)
    i=i+1

  i=0
  file.seek(start2)
  data = file.read(1)
  while  i<32768: #128*256
    try:
      if data != b"" and file.tell() != end1 :
        binaryValues.append(ord(data))
      else :
         binaryValues.append(0)
    except TypeError:
      pass
    data = file.read(1)
    i=i+1

  print(".'idata' and '.edata' are produced.")

  return binaryValues

In [56]:
def createImage(dataSet_t,dataSet_d,dataSet_ie,outputfilename):
    size = len(dataSet_t)
    width = 256

    height = int(size / width) + 1
    image = Image.new("RGB", (width, height))
    data=list(zip(dataSet_t,dataSet_d,dataSet_ie))
    image.putdata(data)
    #image=image.resize((256,256))

    imagename = outputfilename + ".png"
    image.save(imagename)
    image.show()
    print(imagename+" image created")

In [61]:
if __name__ == "__main__":
    files_path = "/content/drive/MyDrive/test/file"  # vir 파일 폴더 경로
    cnt = 0

    for file in os.listdir(files_path):
        path = os.path.dirname(file)
        base_name = os.path.splitext(os.path.basename(file))[0]

        outputFilename = "/content/drive/MyDrive/test/image/" + base_name   
        print(outputFilename)
        
        startAddr_t, endAddr_t, startAddr_d, endAddr_d, startAddr_i, endAddr_i, startAddr_e, endAddr_e = getAddressfromPE(files_path + "/" + file)
        print(startAddr_t, endAddr_t, startAddr_d, endAddr_d, startAddr_i, endAddr_i, startAddr_e, endAddr_e)

        if endAddr_t != 0:
            binaryData_t = getBinaryData_t(files_path + "/" + file, startAddr_t, endAddr_t)
            binaryData_d = getBinaryData_d(files_path + "/" + file, startAddr_d, endAddr_d)
            binaryData_ie = getBinaryData_ie(files_path + "/" + file, startAddr_i, endAddr_i, startAddr_e, endAddr_e)
            createImage(binaryData_t, binaryData_d, binaryData_ie, outputFilename)

/content/drive/MyDrive/test/image/ffd11443a9cbe44f01d097bc996487f9
8192 851968 0 0 0 0 0 0
'.text' data is produced.
'.data' data is produced.
.'idata' and '.edata' are produced.
/content/drive/MyDrive/test/image/ffd11443a9cbe44f01d097bc996487f9.png image created
/content/drive/MyDrive/test/image/ffed7c7e4edb6fc07348d67484283d75
0 0 0 0 0 0 0 0
/content/drive/MyDrive/test/image/ffeae3bebcaf5e3da783b299d794b99f
4096 696320 0 0 0 0 0 0
'.text' data is produced.
'.data' data is produced.
.'idata' and '.edata' are produced.
/content/drive/MyDrive/test/image/ffeae3bebcaf5e3da783b299d794b99f.png image created
/content/drive/MyDrive/test/image/fff8ef9b45bbcb8aca7cb6acdef7d4ba
0 0 0 0 0 0 0 0


In [ ]:
if __name__ == "__main__":
    files_path = "/content/drive/MyDrive/test/file/"  # vir 파일 폴더 경로
    cnt = 0

    for file in os.listdir(files_path):
        cnt += 1
        path = os.path.dirname(file)
        base_name = os.path.splitext(os.path.basename(file))[0]

        train_csv = "E:/trainSet" # csv 파일 경로
        datas = csv.reader(train_csv)

        if cnt % 10 == 0:
            for data in datas:
                if data[0] == file:
                    if data[1] == '0':
                        outputFilename = "E:/testImg" + "/valid/benign/" + base_name  # 저장할 폴더 경로
                    elif data[1] == '1':
                        outputFilename = "E:/testImg" + "/valid/malware/" + base_name
                    break
        else:
            for data in datas:
                if data[0] == file:
                    if data[1] == '0':
                        outputFilename = "E:/testImg" + "/train/benign/" + base_name
                    elif data[1] == '1':
                        outputFilename = "E:/testImg" + "/train/malware/" + base_name
                    break
        train_csv.close()

        startAddr_t, endAddr_t, startAddr_d, endAddr_d, startAddr_i, endAddr_i, startAddr_e, endAddr_e = getAddressfromPE(files_path + "/" + file)

        if endAddr_t != 0:
            binaryData_t = getBinaryData(files_path + "/" + file, startAddr_t, endAddr_t)
            binaryData_d = getBinaryData(files_path + "/" + file, startAddr_d, endAddr_d)
            binaryData_ie = getBinaryData(files_path + "/" + file, startAddr_i, endAddr_i, startAddr_e, endAddr_e)
            createImage(binaryData_t, binaryData_d, binaryData_ie, outputFilename)

        if cnt % 100 == 0:
            print(cnt, " files created!")

In [ ]:
def createImage(dataSet_t,dataSet_d,dataSet_ie,outputfilename):
    size = len(dataSet_t)+len(dataSet_d)+len(dataSet_ie)
    width = 256

    height = int(size / width) + 1
    image = Image.new("RGB", (width, height))
    for data in dataSet_t :
      image.putdata(data)
    for data in dataSet_d :
      image.putdata(data)
    for data in dataSet_ie :
      image.putdata(data)
    image = image.resize((256, 256))

    imagename = outputfilename + ".png"
    image.save(imagename)
    image.show()

In [55]:
def createImage(dataSet_t,dataSet_d,dataSet_ie,outputfilename):
    size = len(dataSet_t)
    width = 256

    height = int(size / width) + 1
    image = Image.new('RGBA', (width, height))
    data=dataSet_t
    data.append(dataSet_d)
    data.append(dataSet_ie)
    image.putdata(data)
    image = image.resize((256, 256))

    imagename = outputfilename + ".png"
    image.save(imagename)
    image.show()
    print(imagename+" image created")